In [15]:
def read_sparse_matrix_method2_crs(filename):
    """
    Read a sparse matrix using Method 2 (Compressed Row Storage - CRS).

    Returns:
    - n: dimension of the matrix
    - values: non-zero values
    - ind_col: column indices
    - row_ptr: row pointers
    """
    with open(filename, "r") as f:
        n = int(f.readline().strip())  # Read dimension

        # Collect entries by row
        entries_by_row = [[] for _ in range(n)]

        for line in f:
            parts = line.strip().split(',')
            if len(parts) == 3:
                value, row, col = float(parts[0]), int(parts[1]), int(parts[2])
                entries_by_row[row].append((col, value))

        # Create CRS arrays
        values = []
        ind_col = []
        row_ptr = [0]

        for row_entries in entries_by_row:
            # Sort entries by column index
            row_entries.sort()

            # Combine entries with the same column index
            combined_entries = {}
            for col, val in row_entries:
                combined_entries[col] = combined_entries.get(col, 0) + val

            # Add entries to values and column indices
            for col, val in sorted(combined_entries.items()):
                values.append(val)
                ind_col.append(col)

            # Update row pointer
            row_ptr.append(len(values))

    return n, values, ind_col, row_ptr

def print_matrix_info_method2_crs(n, values, ind_col, row_ptr):
    """Print information about a sparse matrix stored using Method 2 (CRS)."""
    print(f"Matrix size: {n}x{n}")
    print(f"Number of non-zero elements: {len(values)}")

    # Print first few values, column indices, and row pointers
    print(f"First 5 values: {values[:5]}")
    print(f"First 5 column indices: {ind_col[:5]}")
    print(f"First 5 row pointers: {row_ptr[:5]}")

    # Print number of elements per row for first few rows
    print("Elements per row:")
    for row in range(min(5, n)):
        elements_in_row = row_ptr[row+1] - row_ptr[row]
        print(f"  Row {row}: {elements_in_row} elements")

# Read matrices using revised Method 2 (CRS)
n_a_crs, values_a, ind_col_a, row_ptr_a = read_sparse_matrix_method2_crs("a.txt")
print("Matrix a.txt (CRS format):")
print_matrix_info_method2_crs(n_a_crs, values_a, ind_col_a, row_ptr_a)

n_b_crs, values_b, ind_col_b, row_ptr_b = read_sparse_matrix_method2_crs("b.txt")
print("\nMatrix b.txt (CRS format):")
print_matrix_info_method2_crs(n_b_crs, values_b, ind_col_b, row_ptr_b)

n_aplusb_crs, values_aplusb, ind_col_aplusb, row_ptr_aplusb = read_sparse_matrix_method2_crs("aplusb.txt")
print("\nMatrix aplusb.txt (CRS format):")
print_matrix_info_method2_crs(n_aplusb_crs, values_aplusb, ind_col_aplusb, row_ptr_aplusb)

Matrix a.txt (CRS format):
Matrix size: 2025x2025
Number of non-zero elements: 15087
First 5 values: [162.5, 21.0, 16.5, 23.0, 1.5]
First 5 column indices: [0, 63, 377, 785, 1397]
First 5 row pointers: [0, 5, 18, 22, 32]
Elements per row:
  Row 0: 5 elements
  Row 1: 13 elements
  Row 2: 4 elements
  Row 3: 10 elements
  Row 4: 7 elements

Matrix b.txt (CRS format):
Matrix size: 2025x2025
Number of non-zero elements: 15133
First 5 values: [-162.5, 14.0, 15.0, 173.0, 11.0]
First 5 column indices: [0, 154, 456, 1, 23]
First 5 row pointers: [0, 3, 9, 19, 23]
Elements per row:
  Row 0: 3 elements
  Row 1: 6 elements
  Row 2: 10 elements
  Row 3: 4 elements
  Row 4: 5 elements

Matrix aplusb.txt (CRS format):
Matrix size: 2025x2025
Number of non-zero elements: 28145
First 5 values: [21.0, 14.0, 16.5, 15.0, 23.0]
First 5 column indices: [63, 154, 377, 456, 785]
First 5 row pointers: [0, 6, 24, 37, 50]
Elements per row:
  Row 0: 6 elements
  Row 1: 18 elements
  Row 2: 13 elements
  Row 3: 13

In [16]:
def add_sparse_matrices_method2_crs(n1, values1, ind_col1, row_ptr1, n2, values2, ind_col2, row_ptr2):
    """
    Add two sparse matrices stored in Method 2 (CRS) format.

    Returns:
    - n: dimension of the result matrix
    - values: non-zero values of the result
    - ind_col: column indices of the result
    - row_ptr: row pointers of the result
    """
    # Check dimensions match
    if n1 != n2:
        raise ValueError(f"Matrix dimensions don't match: {n1}x{n1} and {n2}x{n2}")

    n = n1

    # Initialize result
    values_sum = []
    ind_col_sum = []
    row_ptr_sum = [0]

    # Process each row
    for row in range(n):
        # Get elements from first matrix for this row
        start1, end1 = row_ptr1[row], row_ptr1[row+1]
        row_values1 = values1[start1:end1]
        row_cols1 = ind_col1[start1:end1]

        # Get elements from second matrix for this row
        start2, end2 = row_ptr2[row], row_ptr2[row+1]
        row_values2 = values2[start2:end2]
        row_cols2 = ind_col2[start2:end2]

        # Combine elements into a dictionary to handle duplicates
        combined = {}

        # Add elements from first matrix
        for i in range(len(row_cols1)):
            col, val = row_cols1[i], row_values1[i]
            combined[col] = combined.get(col, 0) + val

        # Add elements from second matrix
        for i in range(len(row_cols2)):
            col, val = row_cols2[i], row_values2[i]
            combined[col] = combined.get(col, 0) + val

        # Add combined elements to result, sorted by column
        for col in sorted(combined.keys()):
            val = combined[col]
            # Only include non-zero values
            if abs(val) > 1e-10:  # Using epsilon for floating point comparison
                values_sum.append(val)
                ind_col_sum.append(col)

        # Update row pointer
        row_ptr_sum.append(len(values_sum))

    return n, values_sum, ind_col_sum, row_ptr_sum


def verify_matrix_addition_method2_crs(n_a, values_a, ind_col_a, row_ptr_a,
                                       n_b, values_b, ind_col_b, row_ptr_b,
                                       n_aplusb, values_aplusb, ind_col_aplusb, row_ptr_aplusb,
                                       epsilon=1e-10):
    """
    Verify that matrix a + matrix b = matrix aplusb in CRS format, within epsilon tolerance.
    """
    # First, compute a + b
    n_sum, values_sum, ind_col_sum, row_ptr_sum = add_sparse_matrices_method2_crs(
        n_a, values_a, ind_col_a, row_ptr_a,
        n_b, values_b, ind_col_b, row_ptr_b
    )

    # Check dimensions
    if n_sum != n_aplusb:
        print(f"Dimension mismatch: sum is {n_sum}x{n_sum}, aplusb is {n_aplusb}x{n_aplusb}")
        return False

    # Check number of non-zero elements per row
    for row in range(n_sum):
        elements_sum = row_ptr_sum[row+1] - row_ptr_sum[row]
        elements_aplusb = row_ptr_aplusb[row+1] - row_ptr_aplusb[row]
        if elements_sum != elements_aplusb:
            print(f"Row {row} has different number of elements: sum has {elements_sum}, aplusb has {elements_aplusb}")

            # Optional: print the actual elements for debugging
            start_sum, end_sum = row_ptr_sum[row], row_ptr_sum[row+1]
            start_aplusb, end_aplusb = row_ptr_aplusb[row], row_ptr_aplusb[row+1]

            print(f"Sum row {row} elements: {list(zip(ind_col_sum[start_sum:end_sum], values_sum[start_sum:end_sum]))}")
            print(f"Aplusb row {row} elements: {list(zip(ind_col_aplusb[start_aplusb:end_aplusb], values_aplusb[start_aplusb:end_aplusb]))}")

            return False

    # Check that each element matches
    for row in range(n_sum):
        start_sum, end_sum = row_ptr_sum[row], row_ptr_sum[row+1]
        start_aplusb, end_aplusb = row_ptr_aplusb[row], row_ptr_aplusb[row+1]

        # Get elements from both matrices for this row
        sum_elements = dict(zip(ind_col_sum[start_sum:end_sum], values_sum[start_sum:end_sum]))
        aplusb_elements = dict(zip(ind_col_aplusb[start_aplusb:end_aplusb], values_aplusb[start_aplusb:end_aplusb]))

        # Compare elements
        if sum_elements.keys() != aplusb_elements.keys():
            print(f"Row {row} has different column indices")
            return False

        for col in sum_elements:
            val_sum = sum_elements[col]
            val_aplusb = aplusb_elements[col]
            diff = abs(val_sum - val_aplusb)

            if diff > epsilon:
                print(f"Element at position ({row},{col}) differs by {diff}, which exceeds epsilon={epsilon}")
                return False

    print("Matrix addition verified: a + b = aplusb within the specified epsilon (CRS format)")
    return True

# Test matrix addition and verification with Method 2
n_sum_crs, values_sum, ind_col_sum, row_ptr_sum = add_sparse_matrices_method2_crs(
    n_a_crs, values_a, ind_col_a, row_ptr_a,
    n_b_crs, values_b, ind_col_b, row_ptr_b
)
print("\nComputed sum (a + b) using CRS format:")
print_matrix_info_method2_crs(n_sum_crs, values_sum, ind_col_sum, row_ptr_sum)

# Verify addition using Method 2
print("\nVerifying that a + b = aplusb (CRS format):")
verify_matrix_addition_method2_crs(
    n_a_crs, values_a, ind_col_a, row_ptr_a,
    n_b_crs, values_b, ind_col_b, row_ptr_b,
    n_aplusb_crs, values_aplusb, ind_col_aplusb, row_ptr_aplusb
)


Computed sum (a + b) using CRS format:
Matrix size: 2025x2025
Number of non-zero elements: 28145
First 5 values: [21.0, 14.0, 16.5, 15.0, 23.0]
First 5 column indices: [63, 154, 377, 456, 785]
First 5 row pointers: [0, 6, 24, 37, 50]
Elements per row:
  Row 0: 6 elements
  Row 1: 18 elements
  Row 2: 13 elements
  Row 3: 13 elements
  Row 4: 11 elements

Verifying that a + b = aplusb (CRS format):
✅ Matrix addition verified: a + b = aplusb within the specified epsilon (CRS format)


True

In [17]:
def print_matrix_rows_method2_crs(n, values, ind_col, row_ptr, start_row=0, num_rows=5):
    """Print specific rows of a matrix stored in Method 2 (CRS) format."""
    print(f"Showing rows {start_row} to {min(start_row+num_rows-1, n-1)}:")

    for row in range(start_row, min(start_row+num_rows, n)):
        # Get row bounds
        start_idx = row_ptr[row]
        end_idx = row_ptr[row+1]

        # Extract column indices and values for this row
        row_cols = ind_col[start_idx:end_idx]
        row_vals = values[start_idx:end_idx]

        # Combine into elements list
        elements = list(zip(row_cols, row_vals))

        # Print formatted row
        print(f"Row {row}: {elements}")

# Print the first few rows of the computed sum using Method 2
print("First rows from computed sum (a + b) using Method 2 (CRS):")
print_matrix_rows_method2_crs(n_sum_crs, values_sum, ind_col_sum, row_ptr_sum, 0, 5)

# Print the first few rows from aplusb.txt using Method 2
print("\nFirst rows from aplusb.txt using Method 2 (CRS):")
print_matrix_rows_method2_crs(n_aplusb_crs, values_aplusb, ind_col_aplusb, row_ptr_aplusb, 0, 5)

First rows from computed sum (a + b) using Method 2 (CRS):
Showing rows 0 to 4:
Row 0: [(63, 21.0), (154, 14.0), (377, 16.5), (456, 15.0), (785, 23.0), (1397, 1.5)]
Row 1: [(1, 415.5), (23, 11.0), (55, 10.0), (182, 1.0), (436, 3.5), (463, 23.0), (635, 23.0), (636, 22.5), (652, 11.0), (766, 3.5), (1045, 2.0), (1053, 14.0), (1390, 10.5), (1414, 15.5), (1495, 22.0), (1564, 9.0), (1602, 18.0), (1965, 15.0)]
Row 2: [(2, 329.5), (81, 8.5), (569, 15.5), (601, 14.0), (666, 16.5), (939, 5.5), (1041, 21.5), (1058, 16.0), (1133, 1.0), (1298, 4.0), (1305, 8.5), (1493, 3.0), (1975, 14.5)]
Row 3: [(3, 385.5), (208, 11.5), (308, 20.5), (441, 8.5), (695, 20.5), (1005, 10.0), (1204, 20.0), (1388, 12.0), (1406, 21.0), (1727, 6.0), (1932, 20.5), (1949, 12.5), (1961, 21.5)]
Row 4: [(4, 283.0), (363, 13.0), (626, 8.5), (842, 3.5), (1127, 5.0), (1130, 7.0), (1238, 6.5), (1264, 7.0), (1753, 2.0), (1937, 19.0), (2004, 10.5)]

First rows from aplusb.txt using Method 2 (CRS):
Showing rows 0 to 4:
Row 0: [(63, 2